# Chapter 4 Dynamic Programming

## 4.1 Policy Evaluation (Prediction)
### Exercise 4.1 (Example 4.1 4X4 gridworld)

In [57]:
# rough interfaces
class Agent(object) :
    def __init__(self, env, policy) :
        self.env = env
        self.policy = policy
        self.time = 0
    
    def step(self) :
        current_state = self.env.state()
        action = self.policy.get_action(current_state)
        reward = self.env.action(action)
        new_state = self.env.state()
        self.state_changed(self.time, action, current_state, new_state, reward)
        self.time = self.time + 1

    def state_changed(self, time, action, old_state, new_state, reward) :
        pass
    
class Environment(object) :
    def state(self) :
        pass
    
    def is_terminal_state(self) :
        return False
    
    def action(self, act) : # transit states and return reward
        pass

    def check_dynamics(self, state, act) : # give state, action and get next state and reward
        pass

class Policy(object) :
    def get_action(self, state) :
        pass

In [127]:
import numpy as np
from enum import IntEnum

class Move(IntEnum) :
    UP = 0
    DOWN = 1
    RIGHT = 2
    LEFT = 3
    
    def get_deltas(direction) :
        deltas_dic = { Move.UP : (-1, 0), Move.DOWN : (1, 0), Move.RIGHT : (0, 1), Move.LEFT : (0, -1) }
        return deltas_dic.get(direction)
    
    def get_arrow(direction) :
        arrow_dic = { Move.UP : '↑', Move.DOWN : '↓', Move.RIGHT : '←', Move.LEFT : '→' }
        return arrow_dic.get(direction)

class GridWorld(Environment) :
    def __init__(self, initial_state) :
        self.initialize_states()
        super(Environment, self).__init__()
        self.state = initial_state
        
    def initialize_states(self) :
        self.grid = np.empty((4, 4), dtype=np.int32)
        self.grid[0, 0] = self.grid[3, 3] = -1
        for i in range(1, 15) :
            self.grid[i//4, i - (i//4) * 4] = i
        # print('self.grid:', self.grid)

    def reset_state(self, new_state) :
        self.state = new_state
        
    def state_to_grid(self, state) :
        return (state // 4, state - (state // 4) * 4)
    
    def grid_to_state(self, row, col) :
        return self.grid[row, col]
    
    def is_terminal_state(self) :
        return self.state == -1

    def action(self, act) : # transit states and return reward
        if self.is_terminal_state() :
            print('already reached the terminal state')
            return 0
        
        current_grid = self.state_to_grid(self.state)
        next_grid = np.add(current_grid, Move.get_deltas(act))
        if next_grid[0] < 0 or next_grid[0] >= 4 or next_grid[1] < 0 or next_grid[1] >= 4 :
            print('invalid move')
        else :
            self.state = self.grid_to_state(next_grid[0], next_grid[1])
        return -1

    def check_dynamics(self, state, act) : # give state, action and get next state and reward
        if state == -1 : # terminal state
            return -1, 0
        
        current_grid = self.state_to_grid(state)
        next_grid = np.add(current_grid, Move.get_deltas(act))
        if next_grid[0] < 0 or next_grid[0] >= 4 or next_grid[1] < 0 or next_grid[1] >= 4 :
            next_state = state
        else :
            next_state = self.grid_to_state(next_grid[0], next_grid[1])
        # print('next_grid:', next_grid, ',next_state:', next_state)

        return next_state, -1
    

In [157]:
class GridPolicy(Policy) :
    def __init__(self) :
        self.grid = np.zeros((4, 4, 4), dtype=np.float32)

    def select_greedy(values) :
        indices = np.concatenate(np.where(values == values.max()))
        if len(indices) > 1 : # if we remember prev decision, then we can optimize to select by round-robin
            idx = np.random.randint(0, len(indices))
            return indices[idx]
        return indices[0]

    def _select_greedy_all(values) :
        return np.concatenate(np.where(values == values.max()))
    
    def _state_to_grid(self, state) :
        return (state // 4, state - (state // 4) * 4)

    def values(self, state) :
        return self.grid[self._state_to_grid(state)]

    def set_value(self, state, action, value) :
        self.grid[self._state_to_grid(state)][int(action)] = value

    def set_values(self, state, values) :
        self.grid[self._state_to_grid(state)] = values
        
    def get_action(self, state) :
        values = self.values(state)
        action_idx = GridPolicy.select_greedy(values)
        return Move(action_idx)

    def get_action_indices(self, state) :
        values = self.values(state)
        return GridPolicy._select_greedy_all(values)
    
    def get_actions_str(self, state) :
        values = self.values(state)
        indices = GridPolicy._select_greedy_all(values)
        str = ''
        for idx in indices :
            str = str + Move.get_arrow(Move(idx))
        # print('indices:', indices, ',str:', str)
        return str
    
    def print_policy(self) :
        policy = np.chararray((4, 4), itemsize=4, unicode=True)
        for state in range(1, 15) :
            policy[self._state_to_grid(state)] = self.get_actions_str(state)
        print(policy)

    def print_policy_values(self) :
        for state in range(1, 15) :
            print('state [', state, '], values :', self.grid[self._state_to_grid(state)])



In [129]:
# Policy Evaluation Agent

class GridPolicyEvaluationAgent(Agent) :
    theta = 0.01
    
    def __init__(self, env, policy, gamma=1.0) :
        super(GridPolicyEvaluationAgent, self).__init__(env, policy)
        self.state_values = np.zeros((4, 4), dtype=np.float32)
        self.gamma = gamma

    def _state_to_grid(self, state) :
        return (state // 4, state - (state // 4) * 4)
        
    def state_changed(self, time, action, old_state, new_state, reward) :
        # now we need to update values
        pass

    def run(self) :
        k = 0
        #while True :
        for k in range(30) :
            delta = 0
            for state in range(1, 15) :
                v = self.state_values[self._state_to_grid(state)]
                
                new_v = 0.0
                for action in range(4) :
                    next_state, r = env.check_dynamics(state, Move(action))
                    # print('next state :', next_state)
                    new_v = new_v + 0.25 * (r + self.gamma * self.state_values[self._state_to_grid(next_state)])
                
                self.state_values[self._state_to_grid(state)] = new_v # update in-place
                delta = max(delta, abs(new_v - v))
            if delta < GridPolicyEvaluationAgent.theta :
                print('delta (', delta, ') is less than theta (', GridPolicyEvaluationAgent.theta,
                      '), so breaks the loop...')
                break
            k = k + 1
            print('current state-values (k =', k, ') :')
            print(self.state_values)
        
        #print('final policy values :')
        # self.policy.print_policy_values()
        print('final state-values :')
        print(self.state_values)




In [130]:
env = GridWorld(np.random.randint(1, 14))
policy = GridPolicy()

agent = GridPolicyEvaluationAgent(env, policy, gamma=1.0)
agent.run()

current state-values (k = 1 ) :
[[ 0.        -1.        -1.25      -1.3125   ]
 [-1.        -1.5       -1.6875    -1.75     ]
 [-1.25      -1.6875    -1.84375   -1.8984375]
 [-1.3125    -1.75      -1.8984375  0.       ]]
current state-values (k = 2 ) :
[[ 0.        -1.9375    -2.546875  -2.7304688]
 [-1.9375    -2.8125    -3.2382812 -3.4042969]
 [-2.546875  -3.2382812 -3.5683594 -3.2177734]
 [-2.7304688 -3.4042969 -3.2177734  0.       ]]
current state-values (k = 3 ) :
[[ 0.        -2.8242188 -3.834961  -4.175049 ]
 [-2.8242188 -4.03125   -4.709717  -4.876709 ]
 [-3.834961  -4.709717  -4.963745  -4.264557 ]
 [-4.175049  -4.876709  -4.264557   0.       ]]
current state-values (k = 4 ) :
[[ 0.        -3.6726074 -5.0980835 -5.5812225]
 [-3.6726074 -5.191162  -6.032425  -6.1887283]
 [-5.0980835 -6.032425  -6.148491  -5.150444 ]
 [-5.5812225 -6.1887283 -5.150444   0.       ]]
current state-values (k = 5 ) :
[[ 0.        -4.4904633 -6.3005486 -6.9129305]
 [-4.4904633 -6.261444  -7.224803  -7

In [144]:
# Policy Iteration Agent

class GridPolicyIterationAgent(Agent) :
    theta = 0.01
    
    def __init__(self, env, policy, gamma=1.0) :
        super(GridPolicyIterationAgent, self).__init__(env, policy)
        self.state_values = np.zeros((4, 4), dtype=np.float32)
        self.gamma = gamma

    def _state_to_grid(self, state) :
        return (state // 4, state - (state // 4) * 4)
        
    def state_changed(self, time, action, old_state, new_state, reward) :
        # now we need to update values
        pass

    def eval_policy(self) : 
        k = 0
        #while True :
        for k in range(30) :
            delta = 0
            for state in range(1, 15) :
                v = self.state_values[self._state_to_grid(state)]
                
                new_v = 0.0
                for action in range(4) :
                    next_state, r = env.check_dynamics(state, Move(action))
                    # print('next state :', next_state)
                    new_v = new_v + 0.25 * (r + self.gamma * self.state_values[self._state_to_grid(next_state)])
                
                self.state_values[self._state_to_grid(state)] = new_v # update in-place
                delta = max(delta, abs(new_v - v))
            if delta < GridPolicyEvaluationAgent.theta :
                print('delta (', delta, ') is less than theta (', GridPolicyEvaluationAgent.theta,
                      '), so breaks the loop...')
                break
            k = k + 1
            print('current state-values (k =', k, ') :')
            print(self.state_values)
        
        #print('final policy values :')
        # self.policy.print_policy_values()
        print('final state-values :')
        print(self.state_values)
        
    def improve_policy(self) : # return False if policy is stable (not improved)
        policy_stable = True
        for state in range(1, 15) :
            current_action_indices = policy.get_action_indices(state)
            for action in range(4) :
                next_state, r = env.check_dynamics(state, Move(action))
                # print('next state :', next_state)
                policy.set_value(state, Move(action), r + self.gamma * self.state_values[self._state_to_grid(next_state)])
            new_action_indices = policy.get_action_indices(state)
            if not np.array_equal(current_action_indices, new_action_indices) :
                policy_stable = False
        print('improved policy :')
        policy.print_policy()
        return policy_stable
    
    def run(self) :
        continous_stable_count = 0
        print('initial policy :')
        policy.print_policy()
        for loop in range(30) :
            self.eval_policy()
            if self.improve_policy() :
                print('policy is stable in the loop :', loop)
                continous_stable_count = continous_stable_count + 1
                if continous_stable_count > 2 :
                    break


In [158]:
# create policy
policy = GridPolicy()
agent = GridPolicyIterationAgent(env, policy, gamma=1.0)
agent.run()

initial policy :
[['' '↑↓←→' '↑↓←→' '↑↓←→']
 ['↑↓←→' '↑↓←→' '↑↓←→' '↑↓←→']
 ['↑↓←→' '↑↓←→' '↑↓←→' '↑↓←→']
 ['↑↓←→' '↑↓←→' '↑↓←→' '']]
current state-values (k = 1 ) :
[[ 0.        -1.        -1.25      -1.3125   ]
 [-1.        -1.5       -1.6875    -1.75     ]
 [-1.25      -1.6875    -1.84375   -1.8984375]
 [-1.3125    -1.75      -1.8984375  0.       ]]
current state-values (k = 2 ) :
[[ 0.        -1.9375    -2.546875  -2.7304688]
 [-1.9375    -2.8125    -3.2382812 -3.4042969]
 [-2.546875  -3.2382812 -3.5683594 -3.2177734]
 [-2.7304688 -3.4042969 -3.2177734  0.       ]]
current state-values (k = 3 ) :
[[ 0.        -2.8242188 -3.834961  -4.175049 ]
 [-2.8242188 -4.03125   -4.709717  -4.876709 ]
 [-3.834961  -4.709717  -4.963745  -4.264557 ]
 [-4.175049  -4.876709  -4.264557   0.       ]]
current state-values (k = 4 ) :
[[ 0.        -3.6726074 -5.0980835 -5.5812225]
 [-3.6726074 -5.191162  -6.032425  -6.1887283]
 [-5.0980835 -6.032425  -6.148491  -5.150444 ]
 [-5.5812225 -6.1887283 -5.150

In [154]:
policy = np.chararray((4, 4), itemsize=4, unicode=True)
policy[1, 1] = 'abcd'
print(policy)

[['' '' '' '']
 ['' 'abcd' '' '']
 ['' '' '' '']
 ['' '' '' '']]


In [ ]:
# Exercise 4.3 (equation h/w)


## 4.3 Policy Iteration
### Exercise 4.7 (Example 4.2 Jack's Car Rental)

## 4.4 Value Iteration
### Exercise 4.9 (Example 4.3 Gambler's Problem)

In [ ]:
# Exercise 4.10 (equation h/w)
